In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np

# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")


# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)
# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)


In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

In [ ]:
# CSVファイルを読み込む
file_path = 'data/input/gacha_history.csv'  # ファイルパスを適切に設定してください
df = pd.read_csv(file_path)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df_admin = df[(df["mission_type_id"]==8) | (df["mission_type_id"]==9)]
df_admin

In [ ]:
# 日付をdatetime型に変更
df['mission_achievement_date'] = pd.to_datetime(df['mission_achievement_date'], errors='coerce')
df['get_ticket_date'] = pd.to_datetime(df['get_ticket_date'], errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')

# 4月1日以前は削除、mission_type_idの8と9を削除
df = df[(df["mission_type_id"] != 8) & (df["mission_type_id"] != 9) & (df["mission_achievement_date"] >= pd.Timestamp('2023-04-01'))]
df

In [ ]:
total_tickets_per_user = df.groupby(['user_uid'], observed=True)[
    'add_ticket'].sum()
チケット獲得合計数 = df[df['add_ticket']>0].groupby(['user_uid'], observed=True)[
    'add_ticket'].sum()

# 散布図を描画するためのy座標のみを準備（x座標は必要ない）
y = チケット獲得合計数.values

# 散布図を描画（x座標は全て1として固定）
fig, ax = plt.subplots()
ax.scatter([1] * len(y), y)

ax.set_title('ユーザーごとのチケット合計数')
ax.set_xlabel('ユーザーグループ')
ax.set_ylabel('チケット合計数')

# x軸の目盛りを調整
ax.set_xticks([1])
ax.set_xticklabels(['全ユーザー'])

plt.show()


In [ ]:
# 第一四分位数（Q1）と第三四分位数（Q3）を計算
Q1 = チケット獲得合計数.quantile(0.25)
Q3 = チケット獲得合計数.quantile(0.75)

# 四分位範囲（IQR）を計算
IQR = Q3 - Q1

# 外れ値の範囲を定義
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 外れ値を除外
filtered_data = チケット獲得合計数[(チケット獲得合計数 >= lower_bound) & (チケット獲得合計数 <= upper_bound)]

# ヒストグラムのビンの数を10分割に設定
bin_count = 6

# チケットの最小値と最大値を取得
min_tickets = min(filtered_data)
max_tickets = max(filtered_data)
print(min_tickets,max_tickets)

# ビンの幅を計算
# bin_width = (max_tickets - min_tickets) / bin_count
bin_width = 1

# ビンの範囲を設定
bins = [min_tickets + i * bin_width for i in range(bin_count + 1)]

# ヒストグラムを描画
plt.figure(figsize=(10, 6))
plt.hist(filtered_data, bins=bins, edgecolor='black')
plt.title('ユーザーごとのチケット合計数の分布(外れ値除外)')
plt.xlabel('チケット合計数')
plt.ylabel('ユーザー数')
# plt.xticks(range(min(total_tickets_per_user), max(total_tickets_per_user) + 1))
plt.show()

In [ ]:
# ミッション達成日時ごとの一人当たりの獲得チケット量
average_ticket_by_decade = df.groupby('mission_achievement_date', observed=True)['add_ticket'].mean()
print(average_ticket_by_decade)

# 棒グラフで表示　時間かかるのでコメントアウト
# plt.bar(average_ticket_by_decade.index, average_ticket_by_decade)

In [ ]:
# ミッション達成日時のデータ型を確認し、日付型に変換
df['mission_achievement_date'] = pd.to_datetime(df['mission_achievement_date'])

# ミッション達成日ごとにグループ化して一人当たりの平均チケット量を計算
# 日時データから日付のみを抽出
df['mission_achievement_date'] = df['mission_achievement_date'].dt.date

average_ticket_by_date = df.groupby('mission_achievement_date')['add_ticket'].mean()
# print(average_ticket_by_date)

# 棒グラフで表示
plt.figure(figsize=(10, 6))
plt.bar(average_ticket_by_date.index, average_ticket_by_date)
plt.title('ミッション達成日ごとの一人当たりの平均チケット量')
plt.xlabel('ミッション達成日')
plt.ylabel('平均チケット量')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# mission_type_id 毎の数を算出
# 棒グラフにプロット

n_mission_type_id = df["mission_type_id"].value_counts().sort_index()
print(n_mission_type_id)

・mission_type_idの6（古紙の持込重量ミッション(来月)）,10（チケット条件一致付与(メール)）がない  
・7と13はそもそも存在しない  

・このあたりのデータは削除でよいのではないか

ガチャの効果を測定する  
・ガチャとリサイクル量の相関  
・ログインとリサイクル量の相関

# 以降、結合を試しています

In [ ]:
# user_info とidを紐づけ（試し）
file_path_user_info = 'data/input/user_info_cleansing.csv'  # ファイルパスを適切に設定してください
df_user_info = pd.read_csv(file_path_user_info)
df_user_info.head()

In [ ]:
# birth_dayをdatetimeに変換し、年代を計算
df_user_info['birth_day'] = pd.to_datetime(df_user_info['birth_day'], errors='coerce')
current_year = pd.Timestamp.now().year
df_user_info['age'] = current_year - df_user_info['birth_day'].dt.year
# 年齢と性別が欠損している行を削除
data_age_gender = df_user_info.dropna(subset=['age', 'gender']).copy()
# 年齢を年代に変換
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_user_info['age_group'] = pd.cut(df_user_info['age'], bins=bins, labels=labels, right=False)

In [ ]:
# チケット獲得合計数.name = 'チケット獲得合計数'
tempDf = チケット獲得合計数.to_frame()
tempDf.rename(columns={'add_ticket': 'チケット獲得合計数'}, inplace=True)
tempDf = tempDf.reset_index()

# display(tempDf)

df_user_info = pd.merge(df_user_info, tempDf, left_on='id', right_on='user_uid', how='left')

In [ ]:
# カラムを選択して相関係数を算出
selected_columns = ['id', 'club_coin', 'recycle_point',
       'total_recycle_amount', 'recycle_amount_per_year',
       'recycle_amount_after_gold_member', 'rank_continuation_class','zipcode', 'チケット獲得合計数']
correlation_matrix = df_user_info[selected_columns].corr()

# チケット獲得合計数とリサイクル量・リサイクル頻度の相関を確認するためのヒートマップ
sns.heatmap(correlation_matrix, annot=True)
plt.show()


In [ ]:
# 横軸を対数に変換した散布図行列を表示
df_user_info = df_user_info[df_user_info['チケット獲得合計数'] > 0 ]

# 対数変換を行いたいカラムを選択
columns = ['total_recycle_amount', 'recycle_amount_per_year', 'チケット獲得合計数']

# 対数変換を行うラムダ式を定義し、適用
df_user_info['log_total_recycle_amount'] = df_user_info['total_recycle_amount'].apply(lambda x: np.log10(x) if (x > 0) else x)
df_user_info['log_recycle_amount_per_year'] = df_user_info['recycle_amount_per_year'].apply(lambda x: np.log10(x) if (x > 0) else x)
df_user_info['log_チケット獲得合計数'] = df_user_info['チケット獲得合計数'].apply(lambda x: np.log10(x) if (x > 0) else x)

df_user_info.describe()

#### TODO: リサイクル量ではなくリサイクル回数でも相関出してみよう

In [ ]:
# # 対数変換されたカラムで散布図行列を表示
log_columns = ['log_total_recycle_amount','log_recycle_amount_per_year','log_チケット獲得合計数']
sns.pairplot(df_user_info[log_columns])

常用対数をとったが相関は見られず  
ヒストグラム：totalもper_yearも10^2=100kg が多い  
新聞が1部220gだとすると、1年で約80kg  
段ボール1個150gだとすると、残り20kgすべて段ボールと仮定すると133個/year  
広告・チラシを含めれば年間100kgになるか  
→これに関してSK様に肌感覚的なことも聞きたい  

per_year vs. totalの点がy>xに多い（totalの方がper_yearより多い）

In [ ]:
# merge (左外部結合)
df_merge = pd.merge(df, df_user_info, left_on='user_uid', right_on='id', how='left')

In [ ]:
display(df_merge.head(100))
display(df_merge.describe())
#print(len(df_merge["user_uid"].unique()))
#print(df_merge.info(verbose=True, show_counts=True))
print(df_merge['nickname'].isna().sum())

・user_info は最近やってなかったユーザーを消しているのか？  
・user_info は宮城県で絞ってないか？  
-> user_infoをどのように抽出したかの詳細を聞く必要がある

In [ ]:
# # userIdごとのレコードの出現回数を取得
# record_counts = df_merge.groupby('user_uid_x').size().reset_index(name='record_count')
# record_counts

In [ ]:
# アクティブユーザーと見なされる最小出現回数
threshold = 10

# 'user_id'の出現回数をカウント
user_counts = df_merge['user_uid_x'].value_counts()

# アクティブユーザーを識別
df_merge['アクティブユーザ'] = df_merge['user_uid_x'].apply(lambda x: 1 if (user_counts[x] >= threshold) else 0)


In [ ]:
df_merge.head()

In [ ]:
#columns = ['アクティブユーザ','total_recycle_amount','recycle_amount_per_year', 'チケット獲得合計数']
# total_recycle_amountが５０００以上を除外した場合
#sns.pairplot(df_merge[df_merge['total_recycle_amount']<5000][columns],hue='アクティブユーザ')

# アクティブユーザーごとの相関を出したかったが、うまく定義できず中断
# 一応コードは残してます。

In [ ]:
# 年代ごとのチケット獲得枚数合計を算出
年代ごとのチケット獲得枚数合計 = df_merge[df_merge['add_ticket']>0].groupby('age_group')['add_ticket'].sum()
年代ごとのチケット獲得枚数平均 = df_merge[df_merge['add_ticket']>0].groupby('age_group')['add_ticket'].mean()

In [ ]:
年代ごとのチケット獲得枚数平均

・年齢ごとに大きな差は見受けられなかった。。。

In [ ]:
# 年代ごとのチケット利用枚数合計を算出
年代ごとのチケット利用枚数合計 = df_merge[df_merge['add_ticket']<0].groupby('age_group')['add_ticket'].sum()
年代ごとのチケット利用枚数平均 = df_merge[df_merge['add_ticket']<0].groupby('age_group')['add_ticket'].mean()

In [ ]:
年代ごとのチケット利用枚数平均

In [ ]:
a = df_merge[df_merge['add_ticket']>0].sort_values("user_uid_x")
a[a['user_uid_x']==88]

In [ ]:
# ユーザーごとに分けて獲得枚数履歴を取得

# grouped_age_user = df_merge[(df_merge['add_ticket']>0) & (df_merge['age_group'].notna())].groupby(['age_group', 'user_uid_x'])['add_ticket'].apply(lambda x: x)
grouped_age_user = df_merge[(df_merge['add_ticket']>0) & (df_merge['age_group'].notna())].groupby(['age_group', 'user_uid_x'])['add_ticket'].sum()
grouped_age_user = grouped_age_user.reset_index()
grouped_age_user = grouped_age_user[grouped_age_user['add_ticket'] != 0]

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(x='age_group', y='add_ticket', data=grouped_age_user, showfliers=False)
plt.title('年代ごとのチケット獲得枚数')
plt.xlabel('年代')
plt.ylabel('チケット獲得枚数合計')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# ユーザーごとに分けて利用枚数合計を算出
grouped_age_user = df_merge[(df_merge['add_ticket']<0) & (df_merge['age_group'].notna())].groupby(['age_group', 'user_uid_x'])['add_ticket'].sum()
grouped_age_user = grouped_age_user.reset_index()
grouped_age_user = grouped_age_user[grouped_age_user['add_ticket'] != 0]
grouped_age_user['add_ticket'] = grouped_age_user['add_ticket'] * -1


fig, ax = plt.subplots()

sns.boxplot(x='age_group', y='add_ticket', data=grouped_age_user, showfliers=True)
plt.title('年代ごとのチケット利用枚数')
plt.xlabel('年代')
plt.ylabel('チケット利用枚数合計')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 上のデータを基に最頻値を算出

In [ ]:
# ミッションタイプごとのチケット獲得量の集計
ticket_per_mission_type = df.groupby('mission_type_id')['add_ticket'].sum()
ticket_per_mission_type

In [ ]:
# 年代とミッションタイプごとのチケット獲得量の集計
ticket_per_age_mission = df_merge.groupby(['age_group', 'mission_type_id'], observed=True)['add_ticket'].sum().unstack()
ticket_per_age_mission

In [ ]:
ax = ticket_per_mission_type.plot(kind='bar', figsize=(10, 6))
ax.set_ylabel("チケット獲得量")
ax.set_title("ミッションタイプごとのチケット獲得量")

# 各棒に数値を表示
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
# 図のサイズを調整
plt.figure(figsize=(12, 8)) 

# ヒートマップで年代とミッションタイプごとのチケット獲得量を表示
sns.heatmap(ticket_per_age_mission, annot=True) 
plt.show()

In [ ]:
# 年代ごとのミッションタイプ別チケット獲得割合

df_positive_tickets = df_merge[df_merge['add_ticket'] > 0]

# ステップ1: 年代とミッションタイプごとにチケット獲得量を集計
ticket_per_age_mission = df_positive_tickets.groupby(['age_group', 'mission_type_id'], observed=True)['add_ticket'].sum().unstack(fill_value=0)

# ステップ2: 年代ごとの全ミッションタイプのチケット獲得量の合計を計算
total_tickets_per_age = ticket_per_age_mission.sum(axis=1)

# ステップ3: 各年代内での各ミッションタイプのチケット獲得量の割合を計算
ticket_percentage_per_age = ticket_per_age_mission.div(total_tickets_per_age, axis=0)

# 計算結果の可視化

ax = ticket_percentage_per_age.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('年代ごとのミッションタイプ別チケット獲得割合')
plt.xlabel('年代')
plt.ylabel('チケット獲得割合')
plt.legend(title='ミッションタイプ', bbox_to_anchor=(1.05, 1), loc='upper left')

# 各ミッションタイプごとの割合を注釈として追加
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    if height > 0:
        ax.text(x + width/2, 
                y + height/2, 
                '{:.2g}%'.format(height * 100), 
                horizontalalignment='center', 
                verticalalignment='center')
plt.show()



## ここまでの示唆
・ミッションタイプごとのチケット獲得量は1位：会員サイトにログイン、２位：RPS持ち込み量、３位：ぐるっとポンに新規会員登録<br>
・年代ごとで、利用者が実施したミッションタイプに傾向や特徴はあるか？<br>
　→ミッション12（＝ぐるっとポンに新規会員登録）は割合がわりとバラけている（統計的有意なばらつき）<br>
　→若者(40歳以下)はミッション12の割合が高い傾向？、ミッション12は2023/8/1以降に新規登録したユーザーが対象だから、最近母親世代以下のアプリ利用が増えたのではないか<br>
　→逆にミッション5の会員サイト通算ログインによるチケット獲得割合は、母親世代以下の方が低いし<br>
　→キャンペーンによる付与であるミッションタイプ14も若者世代の割合が高いということで、最近何かキャンペーンをやったのでは<br>

【一応】検定しておく

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
# ミッションタイプ12に焦点を当てた分析を行うためのデータフレーム
mt12_data = df_merge[df_merge['mission_type_id'] == 12]

# 年代ごとのミッションタイプ12のチケット獲得数を集計
mt12_observed = mt12_data.groupby('age_group', observed=True)['add_ticket'].sum()

# 年代ごとの全チケット獲得数を集計
total_observed = df_merge.groupby('age_group', observed=True)['add_ticket'].sum()

# ミッションタイプ12の期待割合を計算（全チケット獲得数に対するミッションタイプ12の割合）
mt12_expected_ratio = mt12_observed.sum() / total_observed.sum()

# 各年代での期待されるミッションタイプ12のチケット獲得数を計算
mt12_expected = total_observed * mt12_expected_ratio

# カイ二乗検定を実行
chi2, p, dof, _ = chi2_contingency(pd.DataFrame({'Observed': mt12_observed, 'Expected': mt12_expected}))

# 結果の出力
print(f"Chi-squared: {chi2:.2f}")
print(f"p-value: {p:.3g}")
print(f"Degrees of freedom: {dof}")

# p値が統計的有意水準（通常0.05）以下かどうかを確認
print("統計的に有意な差がある" if p < 0.05 else "統計的に有意な差がない")

